In [7]:
# 옳지 않은 것 문제들 중 해설 많은 애들 선별하기
import os, json
# from tools.pipeline.config import ONEDRIVE_PATH
ONEDRIVE_PATH = 'C:\\Users\\Jin\\Onedrive\\데이터L\\selectstar'

classified = json.load(open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '7_multiple_rw', 'answer_type_classified.json')))
wrong_data = [p for p in classified if p['answer_type'] == 'wrong']
len(wrong_data)

2500

In [2]:
from tools.core.llm_query import LLMQuery

llm_query = LLMQuery()

def is_full_explanation(llm, question, answer, options, explanation):
    system_prompt = """다음은 문제, 정답, 해설입니다. 해설이 문제의 모든 선지에 대한 설명을 포함하는지 확인해주세요.
    만약 모두 포함하면 'full'을 반환하고, 포함하지 않으면 'notfull'을 반환해주세요.
    """
    user_prompt = f"""
    문제: {question}
    정답: {answer}
    선지: {options}
    해설: {explanation}
    """
    response = llm.query_openrouter(system_prompt, user_prompt, model_name='google/gemini-2.5-flash')
    return response

In [ ]:
wrong_data[0]

In [ ]:
response = is_full_explanation(llm_query, wrong_data[0]['question'], wrong_data[0]['answer'], wrong_data[0]['options'], wrong_data[0]['explanation'])
print(response)

In [3]:
from tqdm import tqdm

full_explanation = []
notfull_explanation = []
fail = []

for wd in tqdm(wrong_data):
    response = is_full_explanation(llm_query, wd['question'], wd['answer'], wd['options'], wd['explanation'])
    if response == 'full':
        full_explanation.append(wd)
    elif response == 'notfull':
        notfull_explanation.append(wd)
    else:
        fail.append(wd)

len(full_explanation)+len(notfull_explanation)+len(fail) == len(wrong_data)

100%|██████████| 2500/2500 [1:02:22<00:00,  1.50s/it]


True

In [7]:
with open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/full_explanation.json'), 'w', encoding='utf-8') as f:
    json.dump(full_explanation, f, ensure_ascii=False, indent=4)

with open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/notfull_explanation.json'), 'w', encoding='utf-8') as f:
    json.dump(notfull_explanation, f, ensure_ascii=False, indent=4)

with open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/fail.json'), 'w', encoding='utf-8') as f:
    json.dump(fail, f, ensure_ascii=False, indent=4)

In [3]:
from tools.pipeline.config import ONEDRIVE_PATH
import os, json

full_explanation = json.load(open(os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '7_multiple_rw', 'full_explanation.json')))
len(full_explanation)

859

In [12]:
q = full_explanation[123]
print(q)

{'file_id': 'SS0397', 'title': '2025 시대에듀 EBS 공인노무사 1차 5개년 기출문제해설', 'chapter': 'PART 02 노동법 Ⅱ', 'tag': 'q_0106_0001', 'domain': '노무', 'subdomain': '노사관계 및 분쟁조정', 'classification_reason': '교원의 노동조합 설립 및 운영, 단체교섭, 조정, 중재 등 노조법 관련 노사관계 조정', 'is_calculation': False, 'question': '교원의 노동조합 설립 및 운영 등에 관한 법률에 관한 설명으로 옳지 않은 것은?', 'options': ['①교원으로 임용되어 근무하였던 사람으로서 노동조합 규약으로 정하는 사람은 노동조합에 가입할 수 있다.', '②전임자는 그 전임기간 중 전임자임을 이유로 승급 또는 그 밖의 신분상의 불이익을 받지 아니한다.', '③단체교섭이 결렬된 경우 중앙노동위원회는 당사자 양쪽이 조정을 신청하는 경우에 한하여 조정을 시작할 수 있다.', '④중앙노동위원회가 제시한 조정안을 당사자의 어느 한쪽이라도 거부한 경우에는 중앙노동위원회는 중재를 한다.', '⑤관계 당사자는 중앙노동위원회의 중재재정이 위법하거나 월권에 의한 것이라고 인정하는 경우에는 중재재정서를 송달받은 날부터 15일 이내에 중앙노동위원회 위원장을 피고로 하여 행정소송을 제기할 수 있다.'], 'answer': '③', 'explanation': '교원의 노동조합 설립 및 운영 등에 관한 법률에 관한 설명으로 옳지 않은 것은?\n① 교원으로 임용되어 근무하였던 사람으로서 노동조합 규약으로 정하는 사람은 노동조합에 가입할 수 있다. 교노법 제4조의2 제2호\n② 전임자는 그 전임기간 중 전임자임을 이유로 승급 또는 그 밖의 신분상의 불이익을 받지 아니한다.\n교노법 제5조 제4항\n❸ 단체교섭이 결렬된 경우 중앙노동위원회는 당사자 양쪽이 조정을 신청하는 경우에 한하여 조정을 시작할 수 있다.\n단체교섭이 결렬된 경우에는 당사자 어

In [13]:
system_prompt = """
다음 지침을 따르며, 주어진 선택지와 해설만으로 객관식 문제를 서술형 문제로 변환하시오.

역할
- 당신은 출제자다. 선택지(선지)와 해설만을 근거로 문제의 주제를 도출하고, 키워드를 포함한 서술형 문제를 만든다.
- 외부 지식이나 원문 질문(문두)은 사용하지 않는다. 선택지와 해설에 나타난 정보만 활용한다.

절차
1) 주제 도출
- 모든 선택지와 해설에서 반복적으로 등장하는 공통 대상을 가장 간결한 명사구로 추출한다.
- 형식: [대상/범주] + [제도/행위/사안] 형태를 우선한다.
- 예: “… 공시해야 한다”, “… 개최 결과 신고” 등이 반복되면 “공시·신고가 수반되는 [행위/제도]”의 정확한 명칭을 선택지 안에서 찾아 정제한다.
- 부정(잘못된 것은 등), 정오 판정, 숫자·기한·빈도 등 정답 단서를 주는 요소는 주제에서 제외한다.

2) 키워드 추출(최대 10개)
- 각 선택지에서 1개씩, 해설에서 필요시 1~2개 추가로, 총 4~10개의 핵심 키워드를 뽑는다.
- 키워드는 문장 대신 2~6어 절의 명사구로 요약한다.
- 수치(예: 반기, 분기, 익일), 임계값, 정오를 드러내는 표현(항상/아니다/잘못 등), 선택지 문장을 그대로 인용하는 행위는 피한다.
- 숫자나 구체치가 있는 경우 범주화하여 일반화한다.
  - 예: “분기마다 개최” → “개최 주기”
  - “익일까지 신고” → “신고 시한”
  - “일시·장소·내용 공시” → “공시 의무/항목”
  - “상장폐지사유” → “제재 또는 불이익 요건”
- 중복 키워드는 병합하고, 주제 이해에 불필요한 세부는 삭제한다.

3) 서술형 문제 생성
- 형태: “다음 키워드를 활용하여 [주제]에 대해 서술하시오.\n키워드: K1, K2, …”
- 키워드는 쉼표로 구분하여 나열한다.
- 정답을 직설적으로 유도하는 표현이나 숫자는 포함하지 않는다.

출력 형식
- 다음 한 줄만 출력한다.
- “다음 키워드를 활용하여 [주제]에 대해 서술하시오.\n키워드: K1, K2, K3, …”

입력 형식(모델에 제공될 내용)
- 선택지: ['① …', '② …', …]
- 해설: …

평가 기준(내부 점검)
- 주제가 선택지·해설의 공통 초점과 일치하는가?
- 키워드가 10개 이하고, 각 키워드가 중립적·범주적 표현인가?
- 숫자·정오 단서를 제거했는가?
- 출력이 지정된 한 줄 형식을 따르는가?
"""

user_prompt = f"""
========= 문제 ========
- 문제 지문: {q['question']}
- 선지 ①~⑤: {q['options']}
- 정답: {q['answer']}
- 선택 해설: {q['explanation']}
"""

In [16]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()

response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/o3')
print(response)

2025-11-19 00:39:30,523 - INFO - Retrying request to /chat/completions in 0.381940 seconds
2025-11-19 00:39:40,917 - INFO - Retrying request to /chat/completions in 0.972773 seconds


다음 키워드를 활용하여 교원노조법상 조합 활동·분쟁 해결 제도에 대해 서술하시오.  
키워드: 가입 대상, 전임자 처우, 단체교섭 결렬, 조정 신청 주체, 중재 개시 요건, 중재재정 불복 소송


In [15]:
q

{'file_id': 'SS0397',
 'title': '2025 시대에듀 EBS 공인노무사 1차 5개년 기출문제해설',
 'chapter': 'PART 02 노동법 Ⅱ',
 'tag': 'q_0106_0001',
 'domain': '노무',
 'subdomain': '노사관계 및 분쟁조정',
 'classification_reason': '교원의 노동조합 설립 및 운영, 단체교섭, 조정, 중재 등 노조법 관련 노사관계 조정',
 'is_calculation': False,
 'question': '교원의 노동조합 설립 및 운영 등에 관한 법률에 관한 설명으로 옳지 않은 것은?',
 'options': ['①교원으로 임용되어 근무하였던 사람으로서 노동조합 규약으로 정하는 사람은 노동조합에 가입할 수 있다.',
  '②전임자는 그 전임기간 중 전임자임을 이유로 승급 또는 그 밖의 신분상의 불이익을 받지 아니한다.',
  '③단체교섭이 결렬된 경우 중앙노동위원회는 당사자 양쪽이 조정을 신청하는 경우에 한하여 조정을 시작할 수 있다.',
  '④중앙노동위원회가 제시한 조정안을 당사자의 어느 한쪽이라도 거부한 경우에는 중앙노동위원회는 중재를 한다.',
  '⑤관계 당사자는 중앙노동위원회의 중재재정이 위법하거나 월권에 의한 것이라고 인정하는 경우에는 중재재정서를 송달받은 날부터 15일 이내에 중앙노동위원회 위원장을 피고로 하여 행정소송을 제기할 수 있다.'],
 'answer': '③',
 'explanation': '교원의 노동조합 설립 및 운영 등에 관한 법률에 관한 설명으로 옳지 않은 것은?\n① 교원으로 임용되어 근무하였던 사람으로서 노동조합 규약으로 정하는 사람은 노동조합에 가입할 수 있다. 교노법 제4조의2 제2호\n② 전임자는 그 전임기간 중 전임자임을 이유로 승급 또는 그 밖의 신분상의 불이익을 받지 아니한다.\n교노법 제5조 제4항\n❸ 단체교섭이 결렬된 경우 중앙노동위원회는 당사자 양쪽이 조정을 신청하는 경우에 한하여 조정을 시작할 수 있다.\n단

In [2]:
import os, json
with open(os.path.join('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json'), 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
len(data)

58482